In [25]:
# install spacy
#!pip install --upgrade spacy
#!pip install spacy-transformers -q
#!python -m spacy download en
#!python -m spacy download en_core_web_lg
#!python -m spacy download en_core_web_trf
#!-m pip install --upgrade transformers
#!python -m spacy download en_trf_bertbaseuncased_lg

In [26]:
import pandas as pd
import numpy as np
from spacy.training.example import Example
import spacy
from spacy.tokens import Doc, Token
import random

In [27]:
data_path = '/content/dataset_short.csv'
df = pd.read_csv(data_path, delimiter=",",  error_bad_lines=False )
print(df.sample(5))
df.info()

      sentence#    word     label
1825       5427      by         O
1537       5392    Gora         O
407        5258     the         O
2694       5540      of         O
3468       5629  Hostra  Mountain
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5134 entries, 0 to 5133
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   sentence#  5134 non-null   int64 
 1   word       5134 non-null   object
 2   label      5134 non-null   object
dtypes: int64(1), object(2)
memory usage: 120.5+ KB


<ipython-input-27-e4dff20f80bd>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(data_path, delimiter=",",  error_bad_lines=False )


In [28]:
nlp = spacy.load('en_core_web_trf')

In [29]:
training_data = []
for sentence_id, group in df.groupby('sentence#'):
    words = group['word'].tolist()
    labels = group['label'].tolist()

    # Convert words to a list of strings
    words = [str(word) for word in words]

    # Create a Doc object from the words
    doc = Doc(nlp.vocab, words=words)

    # Create a list of entities in BIO format
    entities = []
    start = 0
    for word, label in zip(words, labels):
      end = start + len(word)
      # Map all labels other than 'Mountain' to 'O'
      entity_label = 'Mountain' if label == 'Mountain' else 'O'
      entities.append((start, end, doc.vocab.strings[entity_label]))
      start = end + 1  # Assuming words are separated by a single space


    # Add the example to the training data
    example = Example.from_dict(doc, {"entities": entities})
    training_data.append(example)


In [32]:

ner = nlp.get_pipe('ner')


In [34]:
import time
# Update the model with your training data
for epoch in range(1):  # adjust the number of epochs as needed
    # Shuffle the training data
    spacy.util.fix_random_seed(1)
    random.shuffle(training_data)

    # Iterate over batches
    steps = 1
    BATCH_SIZE = 64
    for i, batch in enumerate(spacy.util.minibatch(training_data, size=BATCH_SIZE)):
        step_start = time.time()
        nlp.update(batch)

        step_end = time.time()
        step = step_end - step_start
        steps += step
        avg_time_per_step = steps / (i+1)
        steps_remained = (len(training_data)//BATCH_SIZE - (i))
        estimated = avg_time_per_step * steps_remained

        print(f"{i+1:03d}/{len(training_data)//BATCH_SIZE+1:03d}; step: {round(step, 1)}; ETA: {round(estimated, 1)} seconds", end='\r')
    print()

 010/010; step: 18.3; ETA: -18.9 seconds


In [54]:
text_to_predict = " I'll visit Ukraine in 4 days"
doc = nlp(text_to_predict)

for ent in doc.ents:
    if (ent.label_ == "GPE" or ent.label_ == "LOC" or ent.label_ == "FAC"):
      print(f"Entity: {ent.text}, Label: {ent.label_}")
    else: print(ent.label_)

Entity: Ukraine, Label: GPE
DATE
